# Out-of-State-Contributions: States Analysis

How much out-of-state money have candidates in each of the states raised in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [130]:
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


Import contributions data.

In [131]:
%%notify
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/joe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,13,14,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6130104 entries, 0 to 6130103
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
in_out_state              object
no_veto                   object
office                    object
latest_month              object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(1), int64(2), object(16)
memory usage: 888.6+ MB


<IPython.core.display.Javascript object>

Convert the contribution date and latest month columns to datetime data type.

In [132]:
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["latest_month"] = pd.to_datetime(contributions["latest_month"], errors="coerce")
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6130104 entries, 0 to 6130103
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int64(2), object(14)
memory usage: 888.6+ MB


## Calculate out-of-state contributions by state, office and party

Filter the data by election cycle.

In [133]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_14 = contributions[contributions["year"] == 2014]
contributions_10 = contributions[contributions["year"] == 2010]

Group contributions by state, standardized office, party and in-vs.-out-of-state status and sum contributions.

In [134]:
contributions_by_state_18 = contributions_18.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_18.rename(columns={"amount": "contributions_18"}, inplace=True)
contributions_by_state_14 = contributions_14.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_14.rename(columns={"amount": "contributions_14"}, inplace=True)
contributions_by_state_10 = contributions_10.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_10.rename(columns={"amount": "contributions_10"}, inplace=True)
contributions_by_state_18.head()

,state,standardized_office,party,in_out_state,contributions_18
0,AK,GOVERNOR/LIEUTENANT GOVERNOR,Democratic,in-state,"136,395.14"
1,AK,GOVERNOR/LIEUTENANT GOVERNOR,Democratic,out-of-state,"21,488.99"
2,AK,GOVERNOR/LIEUTENANT GOVERNOR,Republican,in-state,"732,823.45"
3,AK,GOVERNOR/LIEUTENANT GOVERNOR,Republican,out-of-state,"17,484.00"
4,AK,GOVERNOR/LIEUTENANT GOVERNOR,Republican,unknown,"19,158.86"


Pivot dataframe to aggregate each state and party's data in a single row.

In [135]:
contributions_by_state_18 = pd.pivot_table(contributions_by_state_18, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()
contributions_by_state_14 = pd.pivot_table(contributions_by_state_14, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()
contributions_by_state_10 = pd.pivot_table(contributions_by_state_10, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()

It appears that some states have no contributions categorized as "unknown." Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [136]:
contributions_by_state_18.fillna(0, inplace=True)
contributions_by_state_14.fillna(0, inplace=True)
contributions_by_state_10.fillna(0, inplace=True)

Flatten the resulting dataframes' multi-index columns.

In [137]:
contributions_by_state_18.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_18.columns.values]
contributions_by_state_14.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_14.columns.values]
contributions_by_state_10.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_10.columns.values]

Calculate the total contributions by state and office.

In [138]:
contributions_by_state_14

,state,party,contributions_14_governor/lieutenant_governor_in_state,contributions_14_governor/lieutenant_governor_out_of_state,contributions_14_governor/lieutenant_governor_unknown,contributions_14_state_house/assembly_in_state,contributions_14_state_house/assembly_out_of_state,contributions_14_state_house/assembly_unknown,contributions_14_state_senate_in_state,contributions_14_state_senate_out_of_state,contributions_14_state_senate_unknown
0,AK,Democratic,"700,816.58","63,592.07",200.00,"483,592.74","30,548.97",0.00,"230,696.89","11,324.36",0.00
1,AK,Republican,"833,714.36","47,523.00",0.00,"823,846.94","31,657.28",0.00,"733,450.13","15,750.59",0.00
2,AK,Third-Party,"455,022.36","22,776.78",0.00,"36,482.33","1,930.00",0.00,"19,087.02",325.00,0.00
3,AL,Democratic,"221,924.98","4,926.00",0.00,"2,677,674.07","145,411.37","4,602.53","2,152,862.28","229,189.77","1,000.00"
4,AL,Republican,"5,356,915.70","705,138.00","8,000.00","10,809,370.86","572,758.07","10,250.00","10,124,475.59","550,107.46","4,250.00"
5,AL,Third-Party,0.00,0.00,0.00,"11,012.01",0.00,0.00,"232,096.13","4,800.00",0.00
6,AR,Democratic,"4,175,790.56","497,006.59","10,950.45","923,423.23","125,316.00","40,914.13","509,975.28","49,475.00","3,000.00"
7,AR,Republican,"2,333,827.25","407,517.76","6,100.00","1,172,814.26","150,958.25","32,747.90","1,066,301.83","163,030.00","11,849.48"
8,AR,Third-Party,"2,863.74",100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,CA,Democratic,"19,679,376.74","2,836,473.92",0.00,"42,393,363.03","6,082,994.15","5,095.00","13,885,281.90","2,060,100.22","13,402.00"


In [139]:
# 2018
contributions_by_state_18["governor/lieutenant_governor_total"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"]
contributions_by_state_18["state_house/assembly_total"] = contributions_by_state_18["contributions_18_state_house/assembly_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly_unknown"]
contributions_by_state_18["state_senate_total"] = contributions_by_state_18["contributions_18_state_senate_in_state"] + contributions_by_state_18["contributions_18_state_senate_out_of_state"] + contributions_by_state_18["contributions_18_state_senate_unknown"]
# 2014
contributions_by_state_14["governor/lieutenant_governor_total"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"]
contributions_by_state_14["state_house/assembly_total"] = contributions_by_state_14["contributions_14_state_house/assembly_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly_unknown"]
contributions_by_state_14["state_senate_total"] = contributions_by_state_14["contributions_14_state_senate_in_state"] + contributions_by_state_14["contributions_14_state_senate_out_of_state"] + contributions_by_state_14["contributions_14_state_senate_unknown"]
# 2010
contributions_by_state_10["governor/lieutenant_governor_total"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"]
contributions_by_state_10["state_house/assembly_total"] = contributions_by_state_10["contributions_10_state_house/assembly_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly_unknown"]
contributions_by_state_10["state_senate_total"] = contributions_by_state_10["contributions_10_state_senate_in_state"] + contributions_by_state_10["contributions_10_state_senate_out_of_state"] + contributions_by_state_10["contributions_10_state_senate_unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions.

In [140]:
contributions_by_state_18["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_state_house/assembly_in_state"] = contributions_by_state_18["contributions_18_state_house/assembly_in_state"] / (contributions_by_state_18["contributions_18_state_house/assembly_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly_unknown"])
contributions_by_state_18["pct_state_house/assembly_out_of_state"] = contributions_by_state_18["contributions_18_state_house/assembly_out_of_state"] / (contributions_by_state_18["contributions_18_state_house/assembly_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly_unknown"])
contributions_by_state_18["pct_state_house/assembly_unknown"] = contributions_by_state_18["contributions_18_state_house/assembly_unknown"] / (contributions_by_state_18["contributions_18_state_house/assembly_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly_unknown"])
contributions_by_state_18["pct_state_senate_in_state"] = contributions_by_state_18["contributions_18_state_senate_in_state"] / (contributions_by_state_18["contributions_18_state_senate_in_state"] + contributions_by_state_18["contributions_18_state_senate_out_of_state"] + contributions_by_state_18["contributions_18_state_senate_unknown"])
contributions_by_state_18["pct_state_senate_out_of_state"] = contributions_by_state_18["contributions_18_state_senate_out_of_state"] / (contributions_by_state_18["contributions_18_state_senate_in_state"] + contributions_by_state_18["contributions_18_state_senate_out_of_state"] + contributions_by_state_18["contributions_18_state_senate_unknown"])
contributions_by_state_18["pct_state_senate_unknown"] = contributions_by_state_18["contributions_18_state_senate_unknown"] / (contributions_by_state_18["contributions_18_state_senate_in_state"] + contributions_by_state_18["contributions_18_state_senate_out_of_state"] + contributions_by_state_18["contributions_18_state_senate_unknown"])

In [141]:
cols = contributions_by_state_18.columns.tolist()
cols = cols[0:5] + cols[11:12] + cols[5:8] + cols[12:13] + cols[8:11] + cols[13:14] + cols[14:]
contributions_by_state_18 = contributions_by_state_18[cols]
contributions_by_state_18

,state,party,contributions_18_governor/lieutenant_governor_in_state,contributions_18_governor/lieutenant_governor_out_of_state,contributions_18_governor/lieutenant_governor_unknown,governor/lieutenant_governor_total,contributions_18_state_house/assembly_in_state,contributions_18_state_house/assembly_out_of_state,contributions_18_state_house/assembly_unknown,state_house/assembly_total,contributions_18_state_senate_in_state,contributions_18_state_senate_out_of_state,contributions_18_state_senate_unknown,state_senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly_in_state,pct_state_house/assembly_out_of_state,pct_state_house/assembly_unknown,pct_state_senate_in_state,pct_state_senate_out_of_state,pct_state_senate_unknown
0,AK,Democratic,"136,395.14","21,488.99",0.00,"157,884.13","814,061.72","42,857.49","8,780.00","865,699.21","289,060.79","9,572.00",500.00,"299,132.79",0.86,0.14,0.00,0.94,0.05,0.01,0.97,0.03,0.00
1,AK,Republican,"732,823.45","17,484.00","19,158.86","769,466.31","895,598.73","38,028.00",5.00,"933,631.73","400,594.07","20,100.00",0.00,"420,694.07",0.95,0.02,0.02,0.96,0.04,0.00,0.95,0.05,0.00
2,AK,Third-Party,"709,439.48","44,594.00",0.00,"754,033.48","5,912.34",50.00,0.00,"5,962.34",0.00,0.00,0.00,0.00,0.94,0.06,0.00,0.99,0.01,0.00,nan,nan,nan
3,AL,Democratic,"1,912,675.57","78,671.07","-16,485.22","1,974,861.42","2,470,320.93","204,366.95","-10,988.28","2,663,699.60","1,559,772.44","141,819.47","-19,120.00","1,682,471.91",0.97,0.04,-0.01,0.93,0.08,-0.00,0.93,0.08,-0.01
4,AL,Republican,"12,401,895.49","896,446.07","738,622.84","14,036,964.40","7,575,326.21","430,003.69","70,546.24","8,075,876.14","9,375,663.13","693,200.91","-123,032.88","9,945,831.16",0.88,0.06,0.05,0.94,0.05,0.01,0.94,0.07,-0.01
5,AL,Third-Party,0.00,0.00,0.00,0.00,"40,500.00",0.00,0.00,"40,500.00",0.00,0.00,0.00,0.00,nan,nan,nan,1.00,0.00,0.00,nan,nan,nan
6,AR,Democratic,"204,371.47","81,932.52",0.00,"286,303.99","597,205.36","126,651.67","27,053.01","750,910.04","245,193.17","56,749.00",500.00,"302,442.17",0.71,0.29,0.00,0.80,0.17,0.04,0.81,0.19,0.00
7,AR,Republican,"3,786,607.43","399,807.00",0.00,"4,186,414.43","1,476,432.94","142,560.20","56,333.84","1,675,326.98","1,120,896.35","107,695.00","53,550.00","1,282,141.35",0.90,0.10,0.00,0.88,0.09,0.03,0.87,0.08,0.04
8,AR,Third-Party,375.00,0.00,0.00,375.00,"15,345.00",250.00,0.00,"15,595.00",0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.98,0.02,0.00,nan,nan,nan
9,CA,Democratic,"60,117,372.02","6,812,521.35","123,140.98","67,053,034.35","38,616,421.51","7,332,580.04","8,183.14","45,957,184.69","15,396,329.15","2,422,154.48","30,477.25","17,848,960.88",0.90,0.10,0.00,0.84,0.16,0.00,0.86,0.14,0.00


In [142]:
contributions_by_state_14["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_state_house/assembly_in_state"] = contributions_by_state_14["contributions_14_state_house/assembly_in_state"] / (contributions_by_state_14["contributions_14_state_house/assembly_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly_unknown"])
contributions_by_state_14["pct_state_house/assembly_out_of_state"] = contributions_by_state_14["contributions_14_state_house/assembly_out_of_state"] / (contributions_by_state_14["contributions_14_state_house/assembly_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly_unknown"])
contributions_by_state_14["pct_state_house/assembly_unknown"] = contributions_by_state_14["contributions_14_state_house/assembly_unknown"] / (contributions_by_state_14["contributions_14_state_house/assembly_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly_unknown"])
contributions_by_state_14["pct_state_senate_in_state"] = contributions_by_state_14["contributions_14_state_senate_in_state"] / (contributions_by_state_14["contributions_14_state_senate_in_state"] + contributions_by_state_14["contributions_14_state_senate_out_of_state"] + contributions_by_state_14["contributions_14_state_senate_unknown"])
contributions_by_state_14["pct_state_senate_out_of_state"] = contributions_by_state_14["contributions_14_state_senate_out_of_state"] / (contributions_by_state_14["contributions_14_state_senate_in_state"] + contributions_by_state_14["contributions_14_state_senate_out_of_state"] + contributions_by_state_14["contributions_14_state_senate_unknown"])
contributions_by_state_14["pct_state_senate_unknown"] = contributions_by_state_14["contributions_14_state_senate_unknown"] / (contributions_by_state_14["contributions_14_state_senate_in_state"] + contributions_by_state_14["contributions_14_state_senate_out_of_state"] + contributions_by_state_14["contributions_14_state_senate_unknown"])

In [143]:
cols = contributions_by_state_14.columns.tolist()
cols = cols[0:5] + cols[11:12] + cols[5:8] + cols[12:13] + cols[8:11] + cols[13:14] + cols[14:]
contributions_by_state_14 = contributions_by_state_14[cols]
contributions_by_state_14

,state,party,contributions_14_governor/lieutenant_governor_in_state,contributions_14_governor/lieutenant_governor_out_of_state,contributions_14_governor/lieutenant_governor_unknown,governor/lieutenant_governor_total,contributions_14_state_house/assembly_in_state,contributions_14_state_house/assembly_out_of_state,contributions_14_state_house/assembly_unknown,state_house/assembly_total,contributions_14_state_senate_in_state,contributions_14_state_senate_out_of_state,contributions_14_state_senate_unknown,state_senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly_in_state,pct_state_house/assembly_out_of_state,pct_state_house/assembly_unknown,pct_state_senate_in_state,pct_state_senate_out_of_state,pct_state_senate_unknown
0,AK,Democratic,"700,816.58","63,592.07",200.00,"764,608.65","483,592.74","30,548.97",0.00,"514,141.71","230,696.89","11,324.36",0.00,"242,021.25",0.92,0.08,0.00,0.94,0.06,0.00,0.95,0.05,0.00
1,AK,Republican,"833,714.36","47,523.00",0.00,"881,237.36","823,846.94","31,657.28",0.00,"855,504.22","733,450.13","15,750.59",0.00,"749,200.72",0.95,0.05,0.00,0.96,0.04,0.00,0.98,0.02,0.00
2,AK,Third-Party,"455,022.36","22,776.78",0.00,"477,799.14","36,482.33","1,930.00",0.00,"38,412.33","19,087.02",325.00,0.00,"19,412.02",0.95,0.05,0.00,0.95,0.05,0.00,0.98,0.02,0.00
3,AL,Democratic,"221,924.98","4,926.00",0.00,"226,850.98","2,677,674.07","145,411.37","4,602.53","2,827,687.97","2,152,862.28","229,189.77","1,000.00","2,383,052.05",0.98,0.02,0.00,0.95,0.05,0.00,0.90,0.10,0.00
4,AL,Republican,"5,356,915.70","705,138.00","8,000.00","6,070,053.70","10,809,370.86","572,758.07","10,250.00","11,392,378.93","10,124,475.59","550,107.46","4,250.00","10,678,833.05",0.88,0.12,0.00,0.95,0.05,0.00,0.95,0.05,0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,"11,012.01",0.00,0.00,"11,012.01","232,096.13","4,800.00",0.00,"236,896.13",nan,nan,nan,1.00,0.00,0.00,0.98,0.02,0.00
6,AR,Democratic,"4,175,790.56","497,006.59","10,950.45","4,683,747.60","923,423.23","125,316.00","40,914.13","1,089,653.36","509,975.28","49,475.00","3,000.00","562,450.28",0.89,0.11,0.00,0.85,0.12,0.04,0.91,0.09,0.01
7,AR,Republican,"2,333,827.25","407,517.76","6,100.00","2,747,445.01","1,172,814.26","150,958.25","32,747.90","1,356,520.41","1,066,301.83","163,030.00","11,849.48","1,241,181.31",0.85,0.15,0.00,0.86,0.11,0.02,0.86,0.13,0.01
8,AR,Third-Party,"2,863.74",100.00,0.00,"2,963.74",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.97,0.03,0.00,nan,nan,nan,nan,nan,nan
9,CA,Democratic,"19,679,376.74","2,836,473.92",0.00,"22,515,850.66","42,393,363.03","6,082,994.15","5,095.00","48,481,452.18","13,885,281.90","2,060,100.22","13,402.00","15,958,784.12",0.87,0.13,0.00,0.87,0.13,0.00,0.87,0.13,0.00


In [144]:
contributions_by_state_10["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_state_house/assembly_in_state"] = contributions_by_state_10["contributions_10_state_house/assembly_in_state"] / (contributions_by_state_10["contributions_10_state_house/assembly_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly_unknown"])
contributions_by_state_10["pct_state_house/assembly_out_of_state"] = contributions_by_state_10["contributions_10_state_house/assembly_out_of_state"] / (contributions_by_state_10["contributions_10_state_house/assembly_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly_unknown"])
contributions_by_state_10["pct_state_house/assembly_unknown"] = contributions_by_state_10["contributions_10_state_house/assembly_unknown"] / (contributions_by_state_10["contributions_10_state_house/assembly_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly_unknown"])
contributions_by_state_10["pct_state_senate_in_state"] = contributions_by_state_10["contributions_10_state_senate_in_state"] / (contributions_by_state_10["contributions_10_state_senate_in_state"] + contributions_by_state_10["contributions_10_state_senate_out_of_state"] + contributions_by_state_10["contributions_10_state_senate_unknown"])
contributions_by_state_10["pct_state_senate_out_of_state"] = contributions_by_state_10["contributions_10_state_senate_out_of_state"] / (contributions_by_state_10["contributions_10_state_senate_in_state"] + contributions_by_state_10["contributions_10_state_senate_out_of_state"] + contributions_by_state_10["contributions_10_state_senate_unknown"])
contributions_by_state_10["pct_state_senate_unknown"] = contributions_by_state_10["contributions_10_state_senate_unknown"] / (contributions_by_state_10["contributions_10_state_senate_in_state"] + contributions_by_state_10["contributions_10_state_senate_out_of_state"] + contributions_by_state_10["contributions_10_state_senate_unknown"])

In [145]:
cols = contributions_by_state_10.columns.tolist()
cols = cols[0:5] + cols[11:12] + cols[5:8] + cols[12:13] + cols[8:11] + cols[13:14] + cols[14:]
contributions_by_state_10 = contributions_by_state_10[cols]
contributions_by_state_10

,state,party,contributions_10_governor/lieutenant_governor_in_state,contributions_10_governor/lieutenant_governor_out_of_state,contributions_10_governor/lieutenant_governor_unknown,governor/lieutenant_governor_total,contributions_10_state_house/assembly_in_state,contributions_10_state_house/assembly_out_of_state,contributions_10_state_house/assembly_unknown,state_house/assembly_total,contributions_10_state_senate_in_state,contributions_10_state_senate_out_of_state,contributions_10_state_senate_unknown,state_senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly_in_state,pct_state_house/assembly_out_of_state,pct_state_house/assembly_unknown,pct_state_senate_in_state,pct_state_senate_out_of_state,pct_state_senate_unknown
0,AK,Democratic,"633,685.26","38,608.15",-680.00,"671,613.41","623,873.91","24,760.00",376.00,"649,009.91","260,481.58","9,850.00",361.10,"270,692.68",0.94,0.06,-0.00,0.96,0.04,0.00,0.96,0.04,0.00
1,AK,Republican,"1,872,258.31","113,586.30","2,266.62","1,988,111.23","735,867.05","27,277.50",335.00,"763,479.55","303,243.35","6,319.00",758.63,"310,320.98",0.94,0.06,0.00,0.96,0.04,0.00,0.98,0.02,0.00
2,AK,Third-Party,"1,100.00",0.00,0.00,"1,100.00","2,718.37",26.00,0.00,"2,744.37","18,281.00",100.00,0.00,"18,381.00",1.00,0.00,0.00,0.99,0.01,0.00,0.99,0.01,0.00
3,AL,Democratic,"4,941,920.53","424,689.98","2,750.00","5,369,360.51","4,499,897.79","274,129.09","90,728.47","4,864,755.35","6,594,182.01","207,883.28","66,823.13","6,868,888.42",0.92,0.08,0.00,0.92,0.06,0.02,0.96,0.03,0.01
4,AL,Republican,"18,767,593.57","684,498.43","13,680.55","19,465,772.55","4,914,964.06","293,829.26","11,190.76","5,219,984.08","4,748,120.18","231,303.95","11,566.00","4,990,990.13",0.96,0.04,0.00,0.94,0.06,0.00,0.95,0.05,0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,"71,900.00","2,200.00",0.00,"74,100.00","463,864.85","74,900.00","-12,278.82","526,486.03",nan,nan,nan,0.97,0.03,0.00,0.88,0.14,-0.02
6,AR,Democratic,"1,615,583.79","78,100.00",0.00,"1,693,683.79","1,392,665.19","103,719.53","13,018.89","1,509,403.61","1,175,441.33","170,939.40","14,137.79","1,360,518.52",0.95,0.05,0.00,0.92,0.07,0.01,0.86,0.13,0.01
7,AR,Republican,"49,448.32","4,300.00",0.00,"53,748.32","670,587.09","54,325.41","1,515.64","726,428.14","510,852.92","78,196.00",356.00,"589,404.92",0.92,0.08,0.00,0.92,0.07,0.00,0.87,0.13,0.00
8,AR,Third-Party,0.00,0.00,0.00,0.00,"2,681.25",0.00,0.00,"2,681.25",0.00,0.00,0.00,0.00,nan,nan,nan,1.00,0.00,0.00,nan,nan,nan
9,CA,Democratic,"35,235,594.21","2,800,933.56",650.00,"38,037,177.77","33,501,572.49","3,646,998.93","6,850.00","37,155,421.42","15,318,916.95","3,263,569.39",200.00,"18,582,686.34",0.93,0.07,0.00,0.90,0.10,0.00,0.82,0.18,0.00


## Export the data

In [146]:
writer = pd.ExcelWriter("data/states_analysis.xlsx")
contributions_by_state_18.to_excel(writer, "contributions_by_state_18", index=False)
contributions_by_state_14.to_excel(writer, "contributions_by_state_14", index=False)
contributions_by_state_10.to_excel(writer, "contributions_by_state_10", index=False)
writer.save()